In [1]:
import pandas as pd
import numpy as np
import re
from google.cloud import bigquery
import requests

In [1]:
import groupproject

ModuleNotFoundError: No module named 'groupproject'

In [2]:
!pip install google-cloud-bigquery pandas

In [3]:
!pip install db-dtypes

In [4]:
pip show db-dtypes

Name: db-dtypes
Version: 1.4.1
Summary: Pandas Data Types for SQL systems (BigQuery, Spanner)
Home-page: https://github.com/googleapis/python-db-dtypes-pandas
Author: The db-dtypes Authors
Author-email: googleapis-packages@google.com
License: 
Location: D:\anaconda3\Lib\site-packages
Requires: numpy, packaging, pandas, pyarrow
Required-by: 
Note: you may need to restart the kernel to use updated packages.


#Fetching Data from BigQuery.

In [5]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\Santanu Pal\Downloads\biqquery-demo-451221-9b98e9b6eb70.json"

In [6]:
client = bigquery.Client()

In [7]:
query = """
select * from `edhec-business-manageme.luxurydata2502.price-monitoring-2022`
where brand ='Chaumet'
"""

In [8]:
query_job = client.query(query)  # Execute query
df1 = query_job.to_dataframe()  # Convert result to Pandas DataFrame

D:\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [9]:
print(df1.head())

      uid    brand                                                url  price  \
0  734976  Chaumet  https://www.chaumet.com/zh_hans/montre-torsade...   <NA>   
1  734977  Chaumet  https://www.chaumet.com/zh_hans/montre-a-secre...   <NA>   
2  734978  Chaumet  https://www.chaumet.com/zh_hans/montre-josephi...  89110   
3  734979  Chaumet  https://www.chaumet.com/zh_hans/chaumet-montre...  56330   
4  734980  Chaumet  https://www.chaumet.com/zh_hans/montre-josephi...  60110   

  currency                                          image_url  \
0      CNY  https://asset.chaumet.com/m/6466a10f99c1a6f4/o...   
1      CNY  https://asset.chaumet.com/m/4afe2a432ff92cbd/o...   
2      CNY  https://asset.chaumet.com/m/7449ae4f712dd144/o...   
3      CNY  https://asset.chaumet.com/m/4ee9ff91e12af6c1/o...   
4      CNY  https://asset.chaumet.com/m/7dd24ab7074b95b4/o...   

                                collection reference_code country  \
0                MONTRE TORSADE DE CHAUMET     W84772-001  

In [10]:
df1_1 = df1.copy()

In [11]:
print(df1.columns)

Index(['uid', 'brand', 'url', 'price', 'currency', 'image_url', 'collection',
       'reference_code', 'country', 'life_span_date', 'life_span',
       'price_before', 'price_difference', 'price_percent_change',
       'price_changed', 'is_new'],
      dtype='object')


#Fetching Data from CSV

In [12]:
# Read the CSV file with the correct delimiter
df2 = pd.read_csv(r"C:\Users\Santanu Pal\Downloads\PM_extract_Jan_2025_10_brands.csv", sep=';', on_bad_lines='skip')
# Filter the DataFrame to include only rows where the Brand is 'Chaumet'
df3 = df2[df2['brand'] == 'Chaumet']
# Display the first few rows of the filtered DataFrame
print(df3.head())

        brand                                                url  \
1561  Chaumet  https://www.chaumet.com/ae_en/hortensia-watch-...   
1562  Chaumet  https://www.chaumet.com/ae_en/hortensia-watch-...   
1563  Chaumet  https://www.chaumet.com/ae_en/hortensia-watch-...   
1564  Chaumet  https://www.chaumet.com/ae_en/hortensia-watch-...   
1565  Chaumet  https://www.chaumet.com/ae_en/hortensia-watch-...   

                                              image_url collection  \
1561  https://asset.chaumet.com/m/201926e56c3fba5a/o...  HORTENSIA   
1562  https://asset.chaumet.com/m/20c02d1b0fe4c943/o...  HORTENSIA   
1563  https://asset.chaumet.com/m/606c0893a1101ab1/o...  HORTENSIA   
1564  https://asset.chaumet.com/m/6f91985624524d42/o...  HORTENSIA   
1565  https://asset.chaumet.com/m/19189325508e3dae/o...  HORTENSIA   

     reference_code    price currency               country  \
1561          85410  24000.0      AED  United Arab Emirates   
1562          85411  24000.0      AED  Unite

In [13]:
df3.columns

Index(['brand', 'url', 'image_url', 'collection', 'reference_code', 'price',
       'currency', 'country', 'life_span_date', 'category', 'scrapping_date'],
      dtype='object')

In [14]:
print(df1['reference_code'].dtype)

object


#DataPreprocessing

In [15]:
df1_1['reference_code'] = df1_1['reference_code'].astype(str).str[1:6]
df1_1.head(2)

,uid,brand,url,price,currency,image_url,collection,reference_code,country,life_span_date,life_span,price_before,price_difference,price_percent_change,price_changed,is_new
0,734976,Chaumet,https://www.chaumet.com/zh_hans/montre-torsade...,<NA>,CNY,https://asset.chaumet.com/m/6466a10f99c1a6f4/o...,MONTRE TORSADE DE CHAUMET,84772,None,2021-10-30,Q4 October 2021,NaN,0,0.0,0.0,NaN
1,734977,Chaumet,https://www.chaumet.com/zh_hans/montre-a-secre...,<NA>,CNY,https://asset.chaumet.com/m/4afe2a432ff92cbd/o...,MONTRE À SECRET L'ÉPI DE BLÉ DE CHAUMET,84762,None,2021-10-30,Q4 October 2021,NaN,0,0.0,0.0,NaN


In [16]:
df_grouped = df3.groupby(['category', 'reference_code']).size().reset_index(name='count')

# Display the grouped DataFrame
print(df_grouped)

   category reference_code  count
0   Watches          13511      9
1   Watches          21182      9
2   Watches          21183      9
3   Watches          21196      9
4   Watches          83798      9
5   Watches          83878      9
6   Watches          83887      9
7   Watches          84017      9
8   Watches          84028      9
9   Watches          84045      9
10  Watches          84063      9
11  Watches          84207      9
12  Watches          84217      9
13  Watches          84254      9
14  Watches          84420      9
15  Watches          84499      9
16  Watches          84530      9
17  Watches          84755      9
18  Watches          84762      9
19  Watches          84772      9
20  Watches          84826      9
21  Watches          85165      9
22  Watches          85166      9
23  Watches          85167      9
24  Watches          85187      9
25  Watches          85193      9
26  Watches          85194      9
27  Watches          85373      9
28  Watches   

In [17]:
# Convert reference_code to string (if needed)
df1_1['reference_code'] = df1_1['reference_code'].astype(str)
df_grouped['reference_code'] = df_grouped['reference_code'].astype(str)

# Function to fetch category based on reference_code
def get_category(ref_code):
    match = df_grouped[df_grouped['reference_code'] == ref_code]['category']
    return match.iloc[0] if not match.empty else 'Unknown'  # Default value if not found

# Apply function to create a new category column in df1_1
df1_1['category'] = df1_1['reference_code'].apply(get_category)

# Display updated DataFrame
print(df1_1)

           uid    brand                                                url  \
0       734976  Chaumet  https://www.chaumet.com/zh_hans/montre-torsade...   
1       734977  Chaumet  https://www.chaumet.com/zh_hans/montre-a-secre...   
2       734978  Chaumet  https://www.chaumet.com/zh_hans/montre-josephi...   
3       734979  Chaumet  https://www.chaumet.com/zh_hans/chaumet-montre...   
4       734980  Chaumet  https://www.chaumet.com/zh_hans/montre-josephi...   
...        ...      ...                                                ...   
22785  3498393  Chaumet  https://www.chaumet.cn/cn_zh/josephine-rondes-...   
22786  3498394  Chaumet  https://www.chaumet.cn/cn_zh/dandy-watch-large...   
22787  3498395  Chaumet  https://www.chaumet.cn/cn_zh/eclosion-de-chaum...   
22788  3498396  Chaumet  https://www.chaumet.cn/cn_zh/esquisse-de-chaum...   
22789  3498397  Chaumet  https://www.chaumet.cn/cn_zh/colombes-creative...   

        price currency                                         

In [18]:
df1_1.groupby(['category', 'reference_code']).size().reset_index(name='count')

,category,reference_code,count
0,Unknown,0315,6
1,Unknown,0611-,546
2,Unknown,0811-,455
3,Unknown,11288,3
4,Unknown,1182-,91
...,...,...,...
814,Watches,84772,3
815,Watches,84826,3
816,Watches,85165,3
817,Watches,85166,3


In [19]:
# Define the common columns for concatenation
selected_columns = ['brand', 'collection', 'reference_code', 'price', 'currency', 'country', 'category']

# Concatenate df1_1 and df3 while keeping only the selected columns
df4 = pd.concat([df1_1[selected_columns], df3[selected_columns]], ignore_index=True)

# Display the concatenated DataFrame
print(df4.head())

     brand                               collection reference_code    price  \
0  Chaumet                MONTRE TORSADE DE CHAUMET          84772     <NA>   
1  Chaumet  MONTRE À SECRET L'ÉPI DE BLÉ DE CHAUMET          84762     <NA>   
2  Chaumet                MONTRE JOSÉPHINE AIGRETTE          85165  89110.0   
3  Chaumet                MONTRE JOSÉPHINE AIGRETTE          85166  56330.0   
4  Chaumet                MONTRE JOSÉPHINE AIGRETTE          85164  60110.0   

  currency country category  
0      CNY    None  Watches  
1      CNY    None  Watches  
2      CNY    None  Watches  
3      CNY    None  Watches  
4      CNY    None  Unknown  


In [20]:
#df5 = pd.read_csv(r"C:\Users\Santanu Pal\Downloads\Chaumet_Data (2022+2025) EUR 2.csv")
#df5.head

In [21]:
df4.isnull().sum()

brand                 0
collection         9461
reference_code        0
price              3036
currency              0
country           22790
category              0
dtype: int64

In [22]:
df4.shape

(23132, 7)

In [23]:
df4['currency'].unique()

array(['CNY', 'EUR', 'JPY', 'AED', 'KRW', 'HKD', 'GBP', 'SGD', 'TWD'],
      dtype=object)

In [24]:
# Define the currency-to-country mapping dictionary
currency_to_country = {
    "CNY": "China",
    "EUR": "Europe",
    "JPY": "Japan",
    "AED": "United Arab Emirates",
    "KRW": "South Korea",
    "HKD": "Hong Kong",
    "GBP": "United Kingdom",
    "SGD": "Singapore",
    "TWD": "Taiwan"
}

# Update the 'country' column based on 'currency' using map()
df4['country'] = df4['currency'].map(currency_to_country)

# Display updated DataFrame
print(df4.head())

     brand                               collection reference_code    price  \
0  Chaumet                MONTRE TORSADE DE CHAUMET          84772     <NA>   
1  Chaumet  MONTRE À SECRET L'ÉPI DE BLÉ DE CHAUMET          84762     <NA>   
2  Chaumet                MONTRE JOSÉPHINE AIGRETTE          85165  89110.0   
3  Chaumet                MONTRE JOSÉPHINE AIGRETTE          85166  56330.0   
4  Chaumet                MONTRE JOSÉPHINE AIGRETTE          85164  60110.0   

  currency country category  
0      CNY   China  Watches  
1      CNY   China  Watches  
2      CNY   China  Watches  
3      CNY   China  Watches  
4      CNY   China  Unknown  


In [25]:
df4.isnull().sum()

brand                0
collection        9461
reference_code       0
price             3036
currency             0
country              0
category             0
dtype: int64

In [26]:
df5 = df4.dropna()
df5.shape

(11690, 7)

#ExchangeRate Updation

In [27]:
# Exchange rate API settings
API_URL = "https://api.exchangerate-api.com/v4/latest/EUR"  
response = requests.get(API_URL)

if response.status_code == 200:
    exchange_rates = response.json()["rates"]  

    # Function to convert price to EUR
    def convert_to_eur(row):
        if pd.isna(row['price']):  # If price is missing, return None
            return None
        if row['currency'] == 'EUR':  # If already in EUR, return the same price
            return row['price']
        if row['currency'] in exchange_rates:  # Convert other currencies to EUR
            return round(row['price'] / exchange_rates[row['currency']], 2)
        return None  # If currency not found, return None

    # Apply conversion function
    df5['price_Eur'] = df5.apply(convert_to_eur, axis=1)

    # Display updated DataFrame
    print(df5.head)
else:
    print("Error fetching exchange rates!")

<bound method NDFrame.head of          brand                 collection reference_code      price currency  \
2      Chaumet  MONTRE JOSÉPHINE AIGRETTE          85165    89110.0      CNY   
3      Chaumet  MONTRE JOSÉPHINE AIGRETTE          85166    56330.0      CNY   
4      Chaumet  MONTRE JOSÉPHINE AIGRETTE          85164    60110.0      CNY   
5      Chaumet  MONTRE JOSÉPHINE AIGRETTE          85168    84430.0      CNY   
6      Chaumet  MONTRE JOSÉPHINE AIGRETTE          85167   207010.0      CNY   
...        ...                        ...            ...        ...      ...   
23126  Chaumet        LES PIERRES DE REVE          84063  1188000.0      TWD   
23127  Chaumet        LES PIERRES DE REVE          84207  1188000.0      TWD   
23128  Chaumet                   MAHARANI          85193  3844000.0      TWD   
23129  Chaumet           OTHER COLLECTION          84530  2569000.0      TWD   
23130  Chaumet                 SOUVERAINE          85194  3972000.0      TWD   

      cou

C:\Users\Santanu Pal\AppData\Local\Temp\ipykernel_12956\572372160.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['price_Eur'] = df5.apply(convert_to_eur, axis=1)


In [28]:
df5.reset_index(drop=True, inplace=True)
df5.head(1)

,brand,collection,reference_code,price,currency,country,category,price_Eur
0,Chaumet,MONTRE JOSÉPHINE AIGRETTE,85165,89110.0,CNY,China,Watches,11709.59


Uploading the Modified Table to BigQuery

In [29]:
!pip install pandas-gbq google-cloud-bigquery

#Uploading to a Bigquery.

In [31]:
project_id = "biqquery-demo-451221"
dataset_id = "groupprojectdata"
table_id = "your_table_name"

# Full table reference
table_full_name = f"{project_id}.{dataset_id}.{table_id}"

# Step 1: Create dataset if it does not exist
dataset_ref = client.dataset(dataset_id, project=project_id)
try:
    client.get_dataset(dataset_ref)  # Check if dataset exists
    print(f"Dataset '{dataset_id}' already exists.")
except Exception:
    print(f"Dataset '{dataset_id}' not found. Creating it now...")
    dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
    dataset.location = "US"  # Set dataset location (Modify if needed)
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset '{dataset_id}' created successfully.")

# Step 2: Define Job Configuration
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # Replace table if exists (use "WRITE_APPEND" to append)
    autodetect=True,  # Automatically detects schema from DataFrame
)

# Step 3: Upload DataFrame (df5) to BigQuery
job = client.load_table_from_dataframe(df5, table_full_name, job_config=job_config)
job.result()  # Wait for job to complete

print(f" Data successfully uploaded to {table_full_name}")


Dataset 'groupprojectdata' not found. Creating it now...
Dataset 'groupprojectdata' created successfully.
✅ Data successfully uploaded to biqquery-demo-451221.groupprojectdata.your_table_name
